# Lab 05 - Polars and Uncertainty Measurement

In this lab, we will study columnar data formats, in particular Polars and Parquet

In [ ]:
# Package import
import numpy as np

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Data management imports
import pandas as pd
import polars as pl

# Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Get the data
!gdown https://drive.google.com/uc?id=12o15jDuv1RMc8us7ymzj4W_4QeNVB1ii

Downloading...
From: https://drive.google.com/uc?id=12o15jDuv1RMc8us7ymzj4W_4QeNVB1ii
To: /content/email.txt
100% 277k/277k [00:00<00:00, 18.0MB/s]


## Polars

Polars is a very powerful library, but getting use to the new format can be a bit tricky at first. There is an excellent [Migrating from Pandas](https://docs.pola.rs/user-guide/migration/pandas/) tutorial to get you started. For the lab, let's start by getting the data.

Reading data with Polars is different than with Pandas. First, there are two different types of datasets.
1. A [DataFrame](https://docs.pola.rs/api/python/stable/reference/dataframe/index.html) is columnar data that is stored in-memory. **You must have enough RAM memory to read and write each operation in the data**. So, while the fastest method, for large datasets it will impractical.

2. A [LazyFrame](https://docs.pola.rs/api/python/stable/reference/lazyframe/index.html): This is one of the most powerful structures from Polars. It is a dataset in which the **logic** is stored in memory, and the data is only processed when absolutely necessary, or when directly called (via the `.fetch()` function).

In general, you want to work with LazyFrames whenever possible, but these require more abstraction. So, we will start by working with DataFrames and transition to LazyFrames as our data gets bigger.

The core rule to use Polars: You apply functions sequentially in each step. Let's repeat the analyses from the last lab, now using Polars.

In [ ]:
# Load the data using polars. We remove the time column since it is not useful for our analysis.
emails = pl.read_csv('email.txt', separator='\t').drop('time')

# Display the first few rows of the data
emails.head()

spam,to_multiple,from,cc,sent_email,image,attach,dollar,winner,inherit,viagra,password,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,number
i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,str
0,0,1,0,0,0,0,0,"""no""",0,0,0,11.37,202,1,0,0,0,0,"""big"""
0,0,1,0,0,0,0,0,"""no""",0,0,0,10.504,202,1,0,0,0,1,"""small"""
0,0,1,0,0,0,0,4,"""no""",1,0,0,7.773,192,1,0,0,0,6,"""small"""
0,0,1,0,0,0,0,0,"""no""",0,0,0,13.256,255,1,0,0,0,48,"""small"""
0,0,1,0,0,0,0,0,"""no""",0,0,2,1.231,29,0,0,0,0,1,"""none"""


We can see write away some differences. The first is that Polars by default treats all text files as CSV, so we must give it the separator if it is not a comma. `\t` means a tabular space. The second is the typical way that Polars actions are piped. We add the `.drop('time')` function to our output to drop the column from the dataset. The `.head()` function also now shows the data types, which is very helpful for what we want to do.

We can also show summary statistics of our data with the `.describe()` function.

In [ ]:
# Display the summary statistics of the data
emails.describe()

statistic,spam,to_multiple,from,cc,sent_email,image,attach,dollar,winner,inherit,viagra,password,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,number
str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,"""3921""",3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,3921.0,"""3921"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",0.093599,0.158123,0.999235,0.404489,0.27799,0.048457,0.132874,1.467228,null,0.038001,0.00204,0.108136,10.706586,230.658505,0.695231,0.261413,0.080337,0.001785,6.58429,null
"""std""",0.291307,0.364903,0.027654,2.666424,0.448066,0.450848,0.718518,5.022298,null,0.267899,0.127759,0.959931,14.645786,319.304959,0.460368,0.43946,0.271848,0.04222,51.479871,null
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""no""",0.0,0.0,0.0,0.001,1.0,0.0,0.0,0.0,0.0,0.0,"""big"""
"""25%""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,1.459,34.0,0.0,0.0,0.0,0.0,0.0,null
"""50%""",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,5.856,119.0,1.0,0.0,0.0,0.0,1.0,null
"""75%""",0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,null,0.0,0.0,0.0,14.084,298.0,1.0,1.0,0.0,0.0,4.0,null
"""max""",1.0,1.0,1.0,68.0,1.0,20.0,21.0,64.0,"""yes""",9.0,8.0,28.0,190.087,4022.0,1.0,1.0,1.0,1.0,1236.0,"""small"""


We can repeat our study of categorical variables as we did last time. However, this requires fetching the data (i.e., it is only for DataFrames or LazyFrames that have had `.fetch()` applied).

In [ ]:
# Create crosstab of the number and spam columns
emails.pivot(index='number', on='spam', values='spam', aggregate_function='len')

number,0,1
str,u32,u32
"""big""",495,50
"""small""",2659,168
"""none""",400,149


Let's create dummy variables from the categorical variables.

In [ ]:
# Create a dummy variable for the winner and the numbers column
emails = emails.to_dummies(columns=['winner', 'number'], drop_first=True)
emails.head()

spam,to_multiple,from,cc,sent_email,image,attach,dollar,winner_yes,inherit,viagra,password,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,number_none,number_small
i64,i64,i64,i64,i64,i64,i64,i64,u8,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,u8,u8
0,0,1,0,0,0,0,0,0,0,0,0,11.37,202,1,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,0,0,10.504,202,1,0,0,0,1,0,1
0,0,1,0,0,0,0,4,0,1,0,0,7.773,192,1,0,0,0,6,0,1
0,0,1,0,0,0,0,0,0,0,0,0,13.256,255,1,0,0,0,48,0,1
0,0,1,0,0,0,0,0,0,0,0,2,1.231,29,0,0,0,0,1,1,0


In [ ]:
# How many positive class?

# emails.select('spam').mean() returns a dataframe
# to_numpy() returns a 2-D array
# .item() returns a value

posrate = emails.select('spam').mean().to_numpy().item()
print(f"The positive rate of the sample is {posrate*100:.2f}%")

The positive rate of the sample is 9.36%


## Training a model

Fortunately for us, scikit-learn natively supports Polars. So we can create our model using the same code as before.

In [ ]:
# Split data, train logistic regression. 30% of the data will be reserved as test data.
Xtrain, Xtest, ytrain, ytest = train_test_split(emails.drop('spam'),
                          emails.select('spam'),
                          test_size=0.3,
                          random_state=0,
                          stratify=emails.select('spam') # Stratify requires binary values
                          )

In [ ]:
# The result is a polars DataFrame. No need to transform it to a pandas DataFrame.
Xtrain.head()

to_multiple,from,cc,sent_email,image,attach,dollar,winner_yes,inherit,viagra,password,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,number_none,number_small
i64,i64,i64,i64,i64,i64,i64,u8,i64,i64,i64,f64,i64,i64,i64,i64,i64,i64,u8,u8
0,1,0,1,0,0,0,0,0,0,0,1.673,35,1,0,0,0,0,0,1
0,1,0,1,0,0,0,0,0,0,0,5.995,142,1,1,0,0,0,0,1
0,1,0,1,3,3,0,0,0,0,0,10.812,288,1,1,0,0,4,0,1
0,1,0,1,0,0,0,0,0,0,0,2.546,56,1,0,0,0,0,1,0
1,1,16,1,0,0,0,0,0,0,0,10.954,144,0,0,0,0,2,0,1


`ColumnTransformer` and `LogisticRegression` (as all models) also natively support Polars.

In [ ]:
# Define the features to transform
features_to_transform = ['cc', 'image', 'attach', 'dollar', 'viagra',
             'password', 'num_char', 'line_breaks', 'exclaim_mess']

# Create a ColumnTransformer object that scales the features to transform
transform_numbers = ColumnTransformer([('scaler',
                     StandardScaler(),
                     features_to_transform)],
                     remainder='passthrough',
                     verbose_feature_names_out=False,
                     force_int_remainder_cols=False)

# Create a pipeline that scales the features and trains a logistic regression model
logit_pipe = Pipeline([
    ('scaler', transform_numbers),
    ('logistic_regression',
     LogisticRegression(solver='lbfgs',
     penalty = None,
     max_iter=10000,
     verbose=1,
     random_state=20252201,
     n_jobs=-1,
     class_weight='balanced'))
])

# Train the model
logit_pipe.fit(Xtrain, ytrain.to_numpy().ravel())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Pipeline(steps=[('scaler',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  ['cc', 'image', 'attach',
                                                   'dollar', 'viagra',
                                                   'password', 'num_char',
                                                   'line_breaks',
                                                   'exclaim_mess'])],
                                   verbose_feature_names_out=False)),
                ('logistic_regression',
                 LogisticRegression(class_weight='balanced', max_iter=10000,
                                    n_jobs=-1, penalty=None,
                                    random_state=20252201, verbose=1))])

In [ ]:
# Get the training parameters in a dataframe with the corresponding feature names
training_params = pd.DataFrame(logit_pipe.named_steps['logistic_regression'].coef_,
                columns=Xtrain.columns)

# Get the intercept of the logistic regression model
training_params['intercept'] = logit_pipe.named_steps['logistic_regression'].intercept_
training_params

,to_multiple,from,cc,sent_email,image,attach,dollar,winner_yes,inherit,viagra,...,num_char,line_breaks,format,re_subj,exclaim_subj,urgent_subj,exclaim_mess,number_none,number_small,intercept
0,0.006499,-1.347421,0.676533,-0.390746,1.135503,-0.549439,0.239424,-1.300725,0.608871,-3.077456,...,-14.808293,1.776728,0.196751,-0.412961,-1.759959,0.06224,2.557188,0.239626,-1.075511,7.324254


## Confidence interval for a performance measure over a test set.

Now we have a model and we can calculate some unceretainty measures. Let's start with the most basic: What is the AUC confidence interval?

If we have selected a model and want to calculate the confidence interval, we can use either bootstrapping or cross-validation. Depending on the desired outcome, they are both robust. Let's estimate one for two of our measures: Accuracy and AUC. First, let's calculate these measures for the test set.

In [ ]:
# Calculate the accuracy of the model on the test data
accuracy = logit_pipe.score(Xtest, ytest.to_numpy().ravel())

# Get the predicted probabilities of the test data
yprob = logit_pipe.predict_proba(Xtest)[:, 1]

# Get the predicted classes of the test data
ypred = logit_pipe.predict(Xtest)

# Calculate the AUC of the model
auc = roc_auc_score(ytest, yprob)

# Print the results
print(f"The accuracy of the model is {accuracy*100:.2f}%")
print(f"The AUC of the model is {auc:.3f}")

The accuracy of the model is 76.04%
The AUC of the model is 0.893


### Bootstrap

Now we can bootstrap our yprob data 100 times, and calculate the AUC distribution. Then we can calculate the confidence interval. Normally, we would look for a 1,000 or even 10,000 bootstrap, but for the sake of time we will limit this to 100.

In [ ]:
# Create a bootstrap measurement for accuracy and AUC. We will use 100 bootstraps. Normally we would use 1000 or more.
n_bootstraps = 100
bootstrapped_accuracy = np.zeros(n_bootstraps)
bootstrapped_auc = np.zeros(n_bootstraps)

for i in range(n_bootstraps):
    # Get the indices for the bootstrap sample
    idx = np.random.choice(len(ytest), len(ytest), replace=True)

    # Get the accuracy of the bootstrap sample
    bootstrapped_accuracy[i] = accuracy_score(ytest.to_pandas().iloc[idx], ypred[idx])

    # Get the AUC of the bootstrap sample
    bootstrapped_auc[i] = roc_auc_score(ytest.to_pandas().iloc[idx], yprob[idx])

# Get the differences between the bootstrapped values and the original values
accuracy_diff = bootstrapped_accuracy - accuracy
auc_diff = bootstrapped_auc - auc

# Calculate the 95% confidence interval for the accuracy and AUC
accuracy_ci = np.percentile(accuracy_diff, [2.5, 97.5])
auc_ci = np.percentile(auc_diff, [2.5, 97.5])

# Show the ci bounds
print(accuracy_ci)
print(auc_ci)

[-0.02614698  0.02383178]
[-0.02517538  0.02472064]


In [ ]:
# Print the results. Centre the values around the original values.
print(f"The 95% confidence interval for the accuracy is [{(accuracy - accuracy_ci[1])*100:.2f}%, {(accuracy - accuracy_ci[0])*100:.2f}%]")
print(f"The 95% confidence interval for the AUC is [{(auc - auc_ci[1]):.2f}, {(auc - auc_ci[0]):.2f}]")

The 95% confidence interval for the accuracy is [73.66%, 78.66%]
The 95% confidence interval for the AUC is [0.87, 0.92]


The 95% CI of the accuracy over the test population we used to train the model.

### Cross-Validation

Let's compare the above with cross-validation. In cross-validation, we must calculate the models from scratch. This can also be helpful to estimate parameter variability, which we will also conduct. We can do the same with bootstrap, of course.

So our plan is:

1. Partition the training data into N blocks.
2. Calculate the model over N-1 blocks, and test over the remaining.
3. Store the model parameters and the performances for future measurement.

Fortunately, scikit-learn has a full suite to help us with this. It even implements paralellism for faster convergence time. The functions we will need are [`RepeatedKFold`](https://scikit-learn.org/1.6/modules/generated/sklearn.model_selection.RepeatedKFold.html) to create the logic to split the data, and [`cross_validate`](https://scikit-learn.org/1.6/modules/generated/sklearn.model_selection.cross_validate.html) to actually run the cross validation. Let's create the routine.

In [ ]:
# Redefine the pipeline to eliminate verbose and make it sequential
logit_pipe = Pipeline([
    ('scaler', transform_numbers),
    ('logistic_regression', LogisticRegression(solver='lbfgs',
                          penalty = None,
                          max_iter=10000,
                          verbose=0,
                          random_state=20252201,
                          n_jobs=1,
                          class_weight='balanced'))
])

# Create the RepeatedKFold object for 10-by-10
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=20252201)

# Perform cross-validation
cv_results = cross_validate(logit_pipe, # The pipeline to cross-validate
               emails.drop('spam'), # The features
               emails.select('spam').to_numpy().ravel(), # The target
               cv=cv, # The cross-validation object we created
               scoring=['accuracy', 'roc_auc'], # The metrics we want to calculate
               return_estimator=True, # Return the estimator for each fold. Useful for calculating parameter uncertainty.
               n_jobs=-1 # Use all available cores
               )

With all our efficiency measures (polars and parallel processing), this takes only a few seconds to train. If you run this using Pandas and less efficient methods, you are in for a wait!

Now, let's calculate all of our measures.

In [ ]:
x`

The CI for the model accuracy is [70.86%, 79.77%]
The CI for the AUC of the model is [0.839,0.932]
The mean CV accuracy value is 75.32%
The mean CV AUC value is 0.885


How are the cross-validated values compared to the bootstrap values? We are in the same ballpark, but CV gives us a wider range. This is normal as 100 runs of CV training the model from scratch will lead to a more variable estimate. To get tighter bounds, run a 100-by-10 CV, but that can be very expensive computationally.

Which set do you believe the most? Note that bootstrapping may lead to asymmetrical CIs, but CV will always be symmetrical.

To finish, let's calculate some uncertainty on the parameters using the Central Limit Theorem estimates and our method. This is not *exactly* accurate as there is no full independence of the runs (why?), but the bias of doing so is not huge. I leave as an exercise to calculate uncertainty over a specific prediction.

In [ ]:
# Get the coefficients of the logistic regression model for each fold
coefs = [est.named_steps['logistic_regression'].coef_ for est in cv_results['estimator']]

# Get the intercept of the logistic regression model for each fold
intercepts = [est.named_steps['logistic_regression'].intercept_ for est in cv_results['estimator']]

# Get the mean and standard deviation of the coefficients
coefs_mean = np.mean(coefs, axis=0)
coefs_std = np.std(coefs, axis=0)

# Get the mean and standard deviation of the intercept
intercept_mean = np.mean(intercepts)
intercept_std = np.std(intercepts)

# Get the feature names
feature_names = Xtrain.columns

# Create a Dataframe with three variables: The coefficient names, the mean of the coefficients, and the standard deviation of the coefficients
coefs_df = pd.DataFrame({'feature': feature_names, 'mean': coefs_mean.ravel(), 'std': coefs_std.ravel()})

# Add the mean and standard deviation of the intercept to the DataFrame
coefs_df.loc[len(coefs_df)] = ['intercept', intercept_mean, intercept_std]

# Add columns with the lower and upper bounds of the 95% confidence interval
coefs_df['lower'] = coefs_df['mean'] - 1.96*coefs_df['std']
coefs_df['upper'] = coefs_df['mean'] + 1.96*coefs_df['std']


# Print the results
coefs_df

,feature,mean,std,lower,upper
0,to_multiple,-0.017142,0.029163,-0.074303,0.040018
1,from,-1.071492,0.160579,-1.386227,-0.756758
2,cc,0.666575,0.069271,0.530803,0.802347
3,sent_email,-0.427802,0.064125,-0.553487,-0.302117
4,image,1.045099,0.634169,-0.197872,2.288069
5,attach,-0.699962,0.155495,-1.004733,-0.395191
6,dollar,0.572527,0.117862,0.341517,0.803537
7,winner_yes,-1.475475,0.138799,-1.747521,-1.203429
8,inherit,0.534658,0.121845,0.295841,0.773475
9,viagra,-3.082380,0.133039,-3.343136,-2.821623


What do you see here? What do these numbers tell you? In the assignment you will use this for variable selection, although we will see more sophisticated methods next week.

Now you can calculate the uncertainty of any measure and model you want!